## Source Tables at a Glance

In [55]:
import numpy as np
import pandas as pd

### Orders.csv

In [56]:
orders = pd.read_csv("/Users/leanhe/Desktop/Github/InstacartAnalysis/Data/orders.csv")
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


#### Null Values

In [57]:
null_rows_o = orders[orders.isna().any(axis=1)].copy()
null_rows_o.isna().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

#### Unique Counts

In [58]:
print(orders["order_id"].nunique())
print(orders["user_id"].nunique())

3421083
206209


In [59]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


### Products.csv

In [60]:
products = pd.read_csv("/Users/leanhe/Desktop/Github/InstacartAnalysis/Data/products.csv")
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


#### Null Values

In [61]:
null_rows_p = products[products.isna().any(axis=1)].copy()
null_rows_p.isna().sum()

product_id       0.0
product_name     0.0
aisle_id         0.0
department_id    0.0
dtype: float64

### Aisles.csv

In [7]:
aisles = pd.read_csv("/Users/leanhe/Desktop/Github/InstacartAnalysis/Data/aisles.csv")
aisles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


#### Null Values

In [62]:
null_rows_a = aisles[aisles.isna().any(axis=1)].copy()
null_rows_a.isna().sum()

aisle_id    0.0
aisle       0.0
dtype: float64

### Departments.csv

In [63]:
departments = pd.read_csv("/Users/leanhe/Desktop/Github/InstacartAnalysis/Data/departments.csv")
departments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 464.0+ bytes


#### Null Values

In [64]:
null_rows_d = departments[departments.isna().any(axis=1)].copy()
null_rows_d.isna().sum()

department_id    0.0
department       0.0
dtype: float64

### Order_products.csv

In [65]:
order_products = pd.read_csv("/Users/leanhe/Desktop/Github/InstacartAnalysis/Data/order_products.csv")
order_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 989.8 MB


#### Null Values

In [66]:
null_rows_op = order_products[order_products.isna().any(axis=1)].copy()
null_rows_op.isna().sum()

order_id             0.0
product_id           0.0
add_to_cart_order    0.0
reordered            0.0
dtype: float64

## Final Table Preparation

In [29]:
import duckdb

# Connect to an in-memory DuckDB instance
con = duckdb.connect()

In [31]:
con.register("orders", orders)
con.register("order_products", order_products)
con.register("products", products)
con.register("aisles", aisles)
con.register("departments", departments)

In [68]:
metadata = con.execute("""

SELECT
    op.product_id
,   p.product_name
,   op.order_id
,   o.user_id
,   p.aisle_id
,   p.department_id
,   op.reordered
,   o.order_dow

FROM order_products op
JOIN products p ON op.product_id = p.product_id
JOIN orders o ON op.order_id = o.order_id

""").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [73]:
count_df = con.execute("""

SELECT
    product_id
,   product_name
,   COUNT(DISTINCT order_id) as total_order_count
,   COUNT(DISTINCT user_id) as unique_user_count
FROM metadata
GROUP BY 1,2
ORDER BY 3 DESC, 4 DESC

""").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [74]:
count_df.head(10)

,product_id,product_name,total_order_count,unique_user_count
0,24852,Banana,472565,73956
1,13176,Bag of Organic Bananas,379450,63537
2,21137,Organic Strawberries,264683,58838
3,21903,Organic Baby Spinach,241921,55037
4,47209,Organic Hass Avocado,213584,43453
5,47766,Organic Avocado,176815,42771
6,47626,Large Lemon,152657,46402
7,16797,Strawberries,142951,43149
8,26209,Limes,140627,44859
9,27845,Organic Whole Milk,137905,23395


### Pareto Principle Analysis

In [84]:
query = """

WITH base AS(
    SELECT
        product_id
    ,   product_name
    ,   total_order_count
    ,   unique_user_count
    ,   SUM(total_order_count) OVER () AS total_orders
    ,   SUM(unique_user_count) OVER () AS total_users
    FROM count_df
),

ranked AS(
    SELECT
        product_id
    ,   product_name
    ,   total_order_count
    ,   total_orders
    ,   unique_user_count
    ,   total_users
    ,   SUM(total_order_count) OVER (ORDER BY total_order_count DESC) AS cumulative_orders
    ,   SUM(unique_user_count) OVER (ORDER BY unique_user_count DESC) AS cumulative_users
    FROM base
)


    SELECT 
        product_id
    ,   product_name
    ,   total_order_count
    ,   total_orders
    ,   total_users
    ,   cumulative_orders
    ,   cumulative_users
    ,   ROUND(CAST(cumulative_orders AS DOUBLE) /total_orders,4) AS cumulative_order_pct
    ,   ROUND(CAST(cumulative_users AS DOUBLE) /total_users,4) AS cumulative_user_pct
    
    FROM ranked

"""
df_volume_pareto = con.execute(query).fetchdf()

In [85]:
df_volume_pareto.head(10)

,product_id,product_name,total_order_count,total_orders,total_users,cumulative_orders,cumulative_users,cumulative_order_pct,cumulative_user_pct
0,24852,Banana,472565,32434489.0,13307953.0,472565.0,73956.0,0.0146,0.0056
1,13176,Bag of Organic Bananas,379450,32434489.0,13307953.0,852015.0,137493.0,0.0263,0.0103
2,21137,Organic Strawberries,264683,32434489.0,13307953.0,1116698.0,196331.0,0.0344,0.0148
3,21903,Organic Baby Spinach,241921,32434489.0,13307953.0,1358619.0,251368.0,0.0419,0.0189
4,47626,Large Lemon,152657,32434489.0,13307953.0,1901675.0,297770.0,0.0586,0.0224
5,26209,Limes,140627,32434489.0,13307953.0,2185253.0,342629.0,0.0674,0.0257
6,47209,Organic Hass Avocado,213584,32434489.0,13307953.0,1572203.0,386082.0,0.0485,0.0290
7,16797,Strawberries,142951,32434489.0,13307953.0,2044626.0,429231.0,0.0630,0.0323
8,47766,Organic Avocado,176815,32434489.0,13307953.0,1749018.0,472002.0,0.0539,0.0355
9,39275,Organic Blueberries,100060,32434489.0,13307953.0,2888302.0,509140.0,0.0891,0.0383


#### Does ~20% of Instacart products account for ~80% of total orders?

In [87]:
con.execute("""

WITH pareto_order AS (
    SELECT COUNT(*) as top_80_products
    FROM (
        SELECT * 
        FROM df_volume_pareto 
        WHERE cumulative_order_pct <= 0.8
        )
),

total_order AS (
    SELECT COUNT(*) as total_products
    FROM df_volume_pareto
)

SELECT 
    ROUND(CAST(top_80_products AS DOUBLE) / CAST(total_products AS DOUBLE), 4) as pareto_pct_products
FROM pareto_order, total_order
    


""").fetchdf()

,pareto_pct_products
0,0.0913


#### Are those best-selling products also the most widely adopted across users?

In [88]:
con.execute("""

WITH base AS (
    SELECT *
    FROM df_volume_pareto 
    WHERE cumulative_order_pct <= 0.8
)

SELECT
    MAX(cumulative_user_pct) as top_product_user_coverage
FROM base

""").fetchdf()

,top_product_user_coverage
0,0.88


### Saving Top-performing Products' Name into a File for LLM Training

In [90]:
query = """

SELECT DISTINCT 
    product_name
FROM df_volume_pareto
WHERE cumulative_order_pct <= 0.8

"""
top_product_name = con.execute(query).fetchdf()

# Saving it to a .txt file for LLM input

top_product_name['product_name'].to_csv("/Users/leanhe/Desktop/Github/InstacartAnalysis/top_product_names.txt", index=False, header=False)